# Importación de librerías

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import time

%matplotlib inline
%matplotlib notebook
plt.style.use('default')

# Operaciones con los dataframes

### DF 1: Educación de los postulantes

Realizamos un merge de los 3 dataframes de educación de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [2]:
postulantes_educ = pd.read_csv('datos_navent_fiuba/h15_fiuba_1_postulantes_educacion.csv')
postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, how='outer')

postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, how='outer')

# Renombramos la columna 'nombre' por algo mas apropiado
postulantes_educ.rename(columns={'nombre':'titulo_univ'}, inplace=True)

postulantes_educ.head()

,idpostulante,titulo_univ,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [3]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693641 entries, 0 to 693640
Data columns (total 3 columns):
idpostulante    693641 non-null object
titulo_univ     693641 non-null object
estado          693641 non-null object
dtypes: object(3)
memory usage: 21.2+ MB


Transformamos los strings de los títulos universitarios en variables numéricas.
En principio esto lo realizamos para poder eliminar los IDs duplicados del dataframe. Aquí se contemplan los casos de postulantes que ingresaron con distintos títulos o que el estado de su título varió entre una postulación y la otra. A su vez, esto nos servirá más adelante para aplicar los algoritmos de Machine Learning.

Analizamos los distintos tipos de títulos universitarios que existen.

In [4]:
postulantes_educ.groupby('titulo_univ')['titulo_univ'].count()

titulo_univ
Doctorado               620
Master                10074
Otro                  53363
Posgrado              20624
Secundario           244689
Terciario/Técnico    113421
Universitario        250850
Name: titulo_univ, dtype: int64

Pasamos a variables numéricas cada uno de los posibles títulos, a fin de armar un orden de jerarquía de títulos, de manera tal de quedarnos por cada ID del postulante con su título más avanzado.

NOTA: consideramos que 'Otro' es el nivel jerárquico más bajo.

In [5]:
# Otro
postulantes_educ.loc[(postulantes_educ["titulo_univ"] == "Otro"), 'estudios'] = 1
 
# Secundario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario'), 'estudios'] = 2

# Terciario/ Técnico
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico'), 'estudios'] = 3

# Universitario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario'), 'estudios'] = 4

# Posgrado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado'), 'estudios'] = 5

# Master
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master'), 'estudios'] = 6

# Doctorado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado'), 'estudios'] = 7

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios
395401,8MaQejz,Terciario/Técnico,Graduado,3.0
691387,Oqr4QGp,Secundario,Graduado,2.0
271208,5QrzrM,Posgrado,En Curso,5.0
35837,ow5o3Q3,Universitario,Abandonado,4.0
572086,ZDPJNOE,Secundario,Graduado,2.0


Analizamos la cantidad de IDs duplicados

In [6]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

180088

Ordenamos el dataframe según el orden jerárquico de estudio en forma descendente.

In [7]:
postulantes_educ = postulantes_educ.sort_values('estudios', ascending=False)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por orden de estudio, quedará un único ID por postulante con su título más avanzado.

In [8]:
postulantes_educ = postulantes_educ.drop_duplicates('idpostulante')
postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios
450218,NzlaMjL,Secundario,Graduado,2.0
620091,MVOEE8K,Universitario,Abandonado,4.0
443452,EzeMZv6,Universitario,En Curso,4.0
596672,LNweo4q,Universitario,Graduado,4.0
418981,Pm4xGJ0,Terciario/Técnico,Graduado,3.0


Comprobamos que efectivamente no quedaron IDs duplicados.

In [9]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

0

Creamos una nueva columna que indica si el postulante en cuestión tiene un título en curso o ya se graduó/ abandonó

In [10]:
postulantes_educ['estado'].value_counts()

Graduado      253166
En Curso      148362
Abandonado     46381
Name: estado, dtype: int64

In [11]:
postulantes_educ.loc[(postulantes_educ['estado'] == 'Graduado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'Abandonado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'En Curso'), 'esta_estudiando'] = 1

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios,esta_estudiando
153793,lD2pQB8,Secundario,Graduado,2.0,0.0
58223,96948Bw,Universitario,Graduado,4.0,0.0
452117,lD5Z2b8,Secundario,Graduado,2.0,0.0
296137,6aMw1M,Universitario,Graduado,4.0,0.0
6606,lDL6G5z,Secundario,Graduado,2.0,0.0


Eliminamos las columnas 'titulo_univ' y 'estado', ya que la información relativa a esto ya se encuentra contenida en 'estudios' y en 'esta_estudiando'

In [12]:
postulantes_educ.drop(columns={'titulo_univ', 'estado'}, axis=1, inplace=True)
postulantes_educ.sample(5)

,idpostulante,estudios,esta_estudiando
206784,JBX2oKO,4.0,1.0
227476,1Qkd8QE,4.0,1.0
352965,3Nd010X,4.0,0.0
666733,dYjz16M,2.0,0.0
507367,4rdGp3e,2.0,0.0


In [13]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 393408 to 530578
Data columns (total 3 columns):
idpostulante       447909 non-null object
estudios           447909 non-null float64
esta_estudiando    447909 non-null float64
dtypes: float64(2), object(1)
memory usage: 13.7+ MB


### DF 2: Nacimiento y sexo de los postulantes

Realizamos un merge de los 3 dataframes de nacimiento y sexo de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [14]:
postulantes_gen_nac = pd.read_csv('datos_navent_fiuba/h15_fiuba_2_postulantes_genero_y_edad.csv')
postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [15]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505382 entries, 0 to 505381
Data columns (total 3 columns):
idpostulante       505382 non-null object
fechanacimiento    478699 non-null object
sexo               505382 non-null object
dtypes: object(3)
memory usage: 15.4+ MB


Analizamos los posibles valores de la columna 'sexo'

In [16]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

Descartamos los registros que figuran con 'sexo' == 0.0, ya que son una pequeñísima parte del set de datos y no brindan información consistente.

In [17]:
postulantes_gen_nac = postulantes_gen_nac[postulantes_gen_nac['sexo'] != '0.0']

Comprobamos que efectivamente fueron eliminados esos registros

In [18]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
Name: sexo, dtype: int64

Analizamos la cantidad de IDs duplicados

In [19]:
IDs_duplicados = sum(postulantes_gen_nac['idpostulante'].value_counts() > 1)
IDs_duplicados

972

Ordenamos el dataframe según el sexo en forma ascendente.

In [20]:
postulantes_gen_nac = postulantes_gen_nac.sort_values('sexo', ascending=True)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por sexo y que 'NO_DECLARA' tiene un orden alfabético menor que 'FEM' y 'MASC', quedará un único ID por postulante con su información más específica de sexo.

Interpretamos que la información concreta del sexo nos será relevante en posteriores análisis, por eso filtramos los IDs duplicados de esta forma.

In [21]:
postulantes_gen_nac = postulantes_gen_nac.drop_duplicates('idpostulante')
postulantes_gen_nac.sample(5)

,idpostulante,fechanacimiento,sexo
76926,bOJEKld,1994-03-28,FEM
323238,EzeZ299,1992-03-22,NO_DECLARA
346512,bOjAJ5B,1998-02-21,MASC
367199,Ee4PV2,1987-12-04,MASC
370298,12zxL3,1961-06-30,FEM


Se transforma la columna 'fechanacimiento' en 'edad', mediante una serie de operaciones lógicas.

In [22]:
# Renombramos la columna por algo más apropiado
postulantes_gen_nac = postulantes_gen_nac.rename(columns={'fechanacimiento': 'edad'})

# Obtenemos el año de nacimiento
fecha_nac = postulantes_gen_nac['edad'].str.split('-')
año_nac = fecha_nac.str[0]
postulantes_gen_nac['edad'] = año_nac
postulantes_gen_nac['edad'] = pd.to_numeric(postulantes_gen_nac['edad'], errors='coerce').fillna(0).astype(np.int64)

# Filtramos edades que consideramos inválidas
edad_min = postulantes_gen_nac['edad'] > 2000
edad_max = postulantes_gen_nac['edad'] < 1950
condicion_final = ((edad_min | edad_max))
postulantes_gen_nac = postulantes_gen_nac[np.logical_not(condicion_final)]

# Obtenemos la edad real según el año actual
postulantes_gen_nac['edad'] = postulantes_gen_nac['edad'].apply(lambda x: 2018-x)

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo
192846,VNxYv8N,34,FEM
94137,X9xJL5b,26,MASC
188473,X9kE5Qb,38,FEM
495335,mzda6p9,50,MASC
94045,rmRl5Nz,28,FEM


Creamos la columna rango_edad para minimizar la cantidad de valores posibles correspondientes a la edad del postulante.

Rangos:
- 18-25
- 26-30
- 31-40
- 41-68

In [23]:
rango_18_25 = (postulantes_gen_nac['edad'] >= 18) & (postulantes_gen_nac['edad'] <= 25)
rango_26_30 = (postulantes_gen_nac['edad'] >= 26) & (postulantes_gen_nac['edad'] <= 30)
rango_31_40 = (postulantes_gen_nac['edad'] >= 31) & (postulantes_gen_nac['edad'] <= 40)
rango_41_68 = (postulantes_gen_nac['edad'] >= 41) & (postulantes_gen_nac['edad'] <= 68)

postulantes_gen_nac.loc[(rango_18_25), 'rango_edad'] = 1
postulantes_gen_nac.loc[(rango_26_30), 'rango_edad'] = 2
postulantes_gen_nac.loc[(rango_31_40), 'rango_edad'] = 3
postulantes_gen_nac.loc[(rango_41_68), 'rango_edad'] = 4

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo,rango_edad
80457,LNwKzvw,33,FEM,3.0
68150,dYPdYNb,27,MASC,2.0
146390,kPBoXxa,61,MASC,4.0
187700,DrJqjVa,31,MASC,3.0
118992,pzNbdPN,30,MASC,2.0


Eliminamos la columna edad. Consideramos que la información importante se encuentra contenida en el rango edad.

In [24]:
postulantes_gen_nac.drop(columns={'edad'}, axis=1, inplace=True)
postulantes_gen_nac.sample(5)

,idpostulante,sexo,rango_edad
195177,6r595kO,FEM,1.0
369822,PmVvAME,MASC,2.0
199176,4rNbYjM,FEM,2.0
43104,NGVZ54,MASC,3.0
74847,8MZM2aL,FEM,2.0


Analizamos los valores posibles de la columna 'sexo'

In [26]:
postulantes_gen_nac["sexo"].value_counts()

FEM           250194
MASC          226713
NO_DECLARA      1094
Name: sexo, dtype: int64

Pasamos a valor numérico el sexo de los postulantes.

- NO_DECLARA = 0
- FEM = 1
- MASC = 2

In [27]:
postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'NO_DECLARA'), 'sexo'] = 0
postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'FEM'),'sexo'] = 1
postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'MASC'),'sexo'] = 2

postulantes_gen_nac.sample(5)

,idpostulante,sexo,rango_edad
342987,GNXawXp,2,2.0
135041,YjQRmA1,1,4.0
49996,N8MvXL,2,3.0
63287,YB6j0V,2,3.0
386021,zvaA1m5,1,1.0


In [28]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478001 entries, 165770 to 147639
Data columns (total 3 columns):
idpostulante    478001 non-null object
sexo            478001 non-null int64
rango_edad      478001 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 14.6+ MB


Liberamos memoria para mejorar la performance

In [29]:
postulantes_educ_2 = 0
postulantes_gen_nac_2 = 0
fecha_nac = 0
año_nac = 0
edad_min = 0
edad_max = 0
condicion_final = 0
rango_18_25 = 0
rango_26_30 = 0
rango_31_40 = 0
rango_41_68 = 0

### Merge de los DF 1 y 2

Se realiza un merge de los dos dataframes con información correspondiente a los postulantes

In [30]:
postulantes = pd.merge(postulantes_educ, postulantes_gen_nac, on='idpostulante', how='inner')

postulantes.head()

,idpostulante,estudios,esta_estudiando,sexo,rango_edad
0,a0XaWD,7.0,0.0,2,4.0
1,4reGo5z,7.0,0.0,1,2.0
2,X9lpKkb,7.0,0.0,2,4.0
3,RzMXJ4E,7.0,0.0,1,4.0
4,akOA6b9,7.0,0.0,2,4.0


In [31]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438451 entries, 0 to 438450
Data columns (total 5 columns):
idpostulante       438451 non-null object
estudios           438451 non-null float64
esta_estudiando    438451 non-null float64
sexo               438451 non-null int64
rango_edad         438451 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 20.1+ MB


### DF 3: Postulaciones a los avisos

Realizamos un merge de los 2 dataframes de postulaciones a los anuncios que nos fueron provistos, para poder trabajar de manera unificada.

In [3]:
postulaciones = pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones_2 = pd.read_csv('datos_navent_fiuba/h15_fiuba_4_postulaciones.csv')

postulaciones = pd.merge(postulaciones, postulaciones_2, how='outer')

postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


Analizamos la cantidad de registros con los que cuenta el dataframe de postulaciones

In [4]:
registros_totales = len(postulaciones)
registros_totales

6604534

Analizamos la cantidad de postulaciones únicas (de un mismo postulante a un mismo aviso)

In [5]:
registros_unicos = len(postulaciones.drop_duplicates(['idaviso','idpostulante']))
registros_unicos

6603752

In [6]:
registros_duplicados = registros_totales - registros_unicos
registros_duplicados

782

Considerando que hay una diferencia de tan sólo 782 registros (es decir, hay 782 postulaciones duplicadas de un mismo postulante a un mismo aviso), decidimos eliminar los duplicados

In [7]:
postulaciones.drop_duplicates(['idaviso','idpostulante'], inplace=True)

len(postulaciones)

6603752

Eliminamos la columna 'fechapostulacion' ya que no la consideramos relevante para los algoritmos de ML

In [8]:
postulaciones.drop(columns={'fechapostulacion'}, inplace = True)

postulaciones.sample(5)

,idaviso,idpostulante
477467,1112305722,akDjNAN
134610,1112280868,JBrEr5r
2204311,1112305028,MVrm2rL
4126338,1112413043,DrXddxl
6484664,1112447011,ZDrpe0z


Analizamos los avisos top (aquellos que reciben mayor cantidad de postulaciones

In [9]:
cant_postulantes_por_aviso = postulaciones.groupby(['idaviso',]).size().to_frame('cant_postulaciones').reset_index()
cant_postulantes_por_aviso.sort_values('cant_postulaciones', inplace = True, ascending = False)

cant_postulantes_por_aviso.head(10)

,idaviso,cant_postulaciones
10405,1112334791,24420
1084,1112094756,22424
931,1112033906,20570
10402,1112334788,16687
1640,1112196813,13619
6977,1112280937,12124
447,1111753681,11102
1837,1112204682,10098
11224,1112345900,9784
766,1111970596,9746


Agregamos la columna 'sepostulo_x' con valor igual 1 para todos los registros. Esto nos será de gran utilidad a la hora de crear el set de entrenamiento para los algoritmos de ML

In [10]:
postulaciones['sepostulo_x'] = 1

postulaciones.sample(10)

,idaviso,idpostulante,sepostulo_x
2986185,1112292117,LNKwVMr,1
5253028,1112461955,X9k1ZeO,1
930833,1112334806,X9Paap8,1
5432969,1112269661,PmLNbLj,1
751559,1112348683,rmRYJVx,1
4624086,1112385408,1Q4RZxB,1
6476678,1112461848,bOV18qd,1
2377045,1112354925,1b3D8O,1
3661376,1112383417,xkdr256,1
4135749,1112411812,wVkAeWL,1


### DF 4: Detalles de los avisos

Se realiza un merge de los 3 dataframes de detalles de los avisos que nos fueron provistos

In [11]:
avisos_detalles = pd.read_csv('datos_navent_fiuba/h15_fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles.sample(5)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
19050,1112038773,1,Sumate a nuestros nueva edición de Bootcamps F...,"<p><strong>En noviembre, sumate a nuestras cap...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Tecnologia / Sistemas,Globant
5919,1112363369,1,Soporte IT / Help Desk con Inglés Billingüe,"<p style="""">En <strong>Adecco Tecnología</stro...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Soporte Técnico,Adecco -Región Office
14304,1112328960,1,Encargado de Mantenimiento,<p>Importante empresa contratará de manera dir...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Mantenimiento,RANDSTAD
12753,1112454462,1,Encuestador de Calle - Tucuman Capital (Releva...,"<p>Nielsen, empresa multinacional busca:</p><p...",Gran Buenos Aires,NaN,NaN,Full-time,Junior,Marketing,Nielsen
8170,1112306829,1,Ejecutivo Comercial. Canal Distribuidores.,"<p style=""""><span style=""""><span style=""""><spa...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [12]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25563 entries, 0 to 25562
Data columns (total 11 columns):
idaviso                 25563 non-null int64
idpais                  25563 non-null int64
titulo                  25563 non-null object
descripcion             25563 non-null object
nombre_zona             25563 non-null object
ciudad                  137 non-null object
mapacalle               1854 non-null object
tipo_de_trabajo         25563 non-null object
nivel_laboral           25563 non-null object
nombre_area             25563 non-null object
denominacion_empresa    25556 non-null object
dtypes: int64(2), object(9)
memory usage: 2.3+ MB


Eliminamos los avisos duplicados

In [13]:
avisos_detalles = avisos_detalles.drop_duplicates(['idaviso'])

Eliminamos las columnas que consideramos no relevantes. Según la explicación aquí abajo, éstas seran 'idpais', 'ciudad', 'mapacalle', 'descripcion' y 'denominacion_empresa'

In [14]:
# Analizamos la cantidad de 'idpais' posibles
# Dado que hay un único idpais, eliminaremos esta columna.
avisos_detalles['idpais'].value_counts()

1    24950
Name: idpais, dtype: int64

In [15]:
# Analizamos la cantidad de valores nulos en 'ciudad'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['ciudad'].isnull().value_counts()

True     24818
False      132
Name: ciudad, dtype: int64

In [16]:
# Analizamos la cantidad de valores nulos en 'mapacalle'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['mapacalle'].isnull().value_counts()

True     23152
False     1798
Name: mapacalle, dtype: int64

In [17]:
# Eliminamos las columnas arriba mencionadas
avisos_detalles.drop(columns={'idpais', 'ciudad', 'mapacalle', 'descripcion', 'denominacion_empresa'}, axis=1, inplace=True)

avisos_detalles.sample(5)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
9069,1112375329,PL Subgerente de Ventas Hipermercado Zona Pilar,Gran Buenos Aires,Full-time,Jefe / Supervisor / Responsable,Desarrollo de Negocios
7293,1112368642,Analista de Costos Ssr,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Administración
8002,1112240780,Guía de eventos bilingue,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Relaciones Institucionales/Publicas
15446,1112267858,Administrativo de deposito con marcado perfil ...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Administración
23330,1112508525,Vendedores tarjetas de crédito y otros product...,Capital Federal,Part-time,Junior,Ventas


Pasamos a valor numérico el nombre zona, según un orden jerárquico asignado por nosotros. Entendemos que Gran Buenos Aires lidera el orden jerárquico, considerando la gran cantidad de registros con los que cuenta. Luego lo siguen Capital Federal y Otros (todas las zonas restantes)

In [18]:
avisos_detalles['nombre_zona'].value_counts()

Gran Buenos Aires              22972
Capital Federal                 1914
Buenos Aires (fuera de GBA)       42
La Plata                           4
Ciudad de Mendoza                  3
Mendoza                            3
GBA Oeste                          3
Rosario                            2
Cordoba                            1
Tucuman                            1
Catamarca                          1
Santa Cruz                         1
Santa Fe                           1
Neuquen                            1
San Juan                           1
Name: nombre_zona, dtype: int64

In [19]:
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Gran Buenos Aires'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Capital Federal'),'nombre_zona'] = 2
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Buenos Aires (fuera de GBA)'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'GBA Oeste'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'La Plata'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Ciudad de Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Catamarca'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Rosario'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Cruz'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'San Juan'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Cordoba'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Tucuman'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Neuquen'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Fe'),'nombre_zona'] = 1

Pasamos a valor numérico el tipo de trabajo, según un orden jerárquico asignado por nosotros.

In [20]:
avisos_detalles['tipo_de_trabajo'].value_counts()

Full-time          22523
Part-time           1725
Teletrabajo          248
Por Horas            124
Pasantia             119
Temporario            89
Por Contrato          87
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

In [21]:
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Full-time'),'tipo_de_trabajo'] = 10
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Part-time'),'tipo_de_trabajo'] = 9
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Teletrabajo'),'tipo_de_trabajo'] = 8
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Horas'),'tipo_de_trabajo'] = 7
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Pasantia'),'tipo_de_trabajo'] = 6
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Temporario'),'tipo_de_trabajo'] = 5
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Contrato'),'tipo_de_trabajo'] = 4
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Fines de Semana'),'tipo_de_trabajo'] = 3
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Primer empleo'),'tipo_de_trabajo'] = 2
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Voluntario'),'tipo_de_trabajo'] = 1

Pasamos a valor numérico el nivel laboral, según un orden jerárquico asignado por nosotros.

In [22]:
avisos_detalles['nivel_laboral'].value_counts()

Senior / Semi-Senior                    16977
Junior                                   4149
Otro                                     1975
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

In [23]:
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Gerencia / Alta Gerencia / Dirección'),'nivel_laboral'] = 5
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Jefe / Supervisor / Responsable'),'nivel_laboral'] = 4
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Senior / Semi-Senior'),'nivel_laboral'] = 3
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Junior'),'nivel_laboral'] = 2
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Otro'),'nivel_laboral'] = 1

In [24]:
avisos_detalles.sample(5)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
1457,1111364281,Vendedor,3,10,3,Ventas
12777,1112388994,Calderista con carnet habilitante- Wilde,2,10,3,Producción
19978,1112471645,Vendedor comercial,3,10,3,Ventas
21487,1111575970,Representante Comercial Telefónico  Martínez ...,2,10,1,Ventas
12801,1112126932,Ayudante de Cocina / MINUTEROS / Peon de cocina,3,10,3,Gastronomia


In [25]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24950 entries, 0 to 25549
Data columns (total 6 columns):
idaviso            24950 non-null int64
titulo             24950 non-null object
nombre_zona        24950 non-null int64
tipo_de_trabajo    24950 non-null int64
nivel_laboral      24950 non-null int64
nombre_area        24950 non-null object
dtypes: int64(4), object(2)
memory usage: 1.3+ MB


Liberamos memoria para mejorar la performance

In [59]:
lista = 0
fecha = 0
postulaciones_2 = 0
avisos_detalles_2 = 0
postulantes_educ = 0
postulantes_gen_nac = 0

# Modelo Final

A continuación, utilizaremos los dataframes que procesamos en la sección anterior para generar el modelo final que será utilizado como set de entrenamiento por los algoritmos de ML.

Básicamente, utilizamos el dataframe de postulaciones para tener registros con postulaciones efectivas (sepostulo = 1). Luego, para generar postulaciones no efectivas (sepostulo = 0), tomamos todos los avisos para los cuales tenemos detalles y los intersectamos con una cantidad random prefijada de postulantes para los cuales tenemos información.

La siguiente función devuelve una cantidad random de ids de postulantes

In [61]:
def devolver_n_ids(n_ids):
    return np.array(postulantes['idpostulante'].sample(n_ids))

Generamos una lista de tuplas (idaviso, idpostulante) con idpostulante random para todos los avisos. Luego, llevamos esto a un dataframe y agregamos la columna 'sepostulo_y', que nos será de gran utilidad al momento de mergearlo con el dataframe de postulaciones

In [62]:
t0 = time()

ids_avisos = np.array(avisos_detalles['idaviso'])
n_ids = 250

lista_avisos_postulantes = []

for idaviso in ids_avisos:
    ids_postulantes = devolver_n_ids(n_ids)
    for idpostulante in ids_postulantes:
        tupla = (idaviso, idpostulante)
        lista_avisos_postulantes.append(tupla)
        
tf = time() - t0
print("Tiempo de ejecución:", tf)

Tiempo de ejecución: 230.87838983535767


In [63]:
df_avisos_postulantes_all = pd.DataFrame(lista_avisos_postulantes, columns=['idaviso','idpostulante'])
df_avisos_postulantes_all['sepostulo_y'] = 0

df_avisos_postulantes_all.head(10)

,idaviso,idpostulante,sepostulo_y
0,1111556097,EDp099,0
1,1111556097,NzewELB,0
2,1111556097,mzGKvNG,0
3,1111556097,5mlx0mN,0
4,1111556097,6J5YkM,0
5,1111556097,Yajx3Y,0
6,1111556097,ZD8mL51,0
7,1111556097,owzell0,0
8,1111556097,VNe6mDl,0
9,1111556097,YmMm8D,0


In [64]:
df_avisos_postulantes_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6237500 entries, 0 to 6237499
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
sepostulo_y     int64
dtypes: int64(2), object(1)
memory usage: 142.8+ MB


Mergeamos el dataframe arriba obtenido con el de postulaciones. Se realiza un outer join para quedarnos con absolutamente todas las combinaciones de idaviso e idpostulante que generamos. Además, gracias a esto, tendremos valores 0 y 1 para la columna 'sepostulo'.

In [65]:
t0 = time()

modelo = pd.merge(df_avisos_postulantes_all, postulaciones, on=['idaviso','idpostulante'], how='outer')

tf = time() - t0
print("Tiempo de ejecución:", tf)

modelo.head(5)

Tiempo de ejecución: 6.407086610794067


,idaviso,idpostulante,sepostulo_y,sepostulo_x
0,1111556097,EDp099,0.0,NaN
1,1111556097,NzewELB,0.0,NaN
2,1111556097,mzGKvNG,0.0,NaN
3,1111556097,5mlx0mN,0.0,NaN
4,1111556097,6J5YkM,0.0,NaN


Pasamos los valores nulos a cero, para poder realizar las operaciones necesarias

In [66]:
modelo.loc[(modelo['sepostulo_x'].isnull()), 'sepostulo_x'] = 0
modelo.loc[(modelo['sepostulo_y'].isnull()), 'sepostulo_y'] = 0

modelo.head(5)

,idaviso,idpostulante,sepostulo_y,sepostulo_x
0,1111556097,EDp099,0.0,0.0
1,1111556097,NzewELB,0.0,0.0
2,1111556097,mzGKvNG,0.0,0.0
3,1111556097,5mlx0mN,0.0,0.0
4,1111556097,6J5YkM,0.0,0.0


In [67]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12837957 entries, 0 to 12837956
Data columns (total 4 columns):
idaviso         int64
idpostulante    object
sepostulo_y     float64
sepostulo_x     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 489.7+ MB


Finalmente creamos la columna 'sepostulo'. Ésta surge de una suma lógica entre las columnas 'sepostulo_x' y 'sepostulo_y'.

In [68]:
modelo['sepostulo'] = modelo['sepostulo_x'] + modelo['sepostulo_y']
modelo.drop(columns={'sepostulo_x','sepostulo_y'}, inplace=True)

modelo.head(5)

/home/towers/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


Tiempo de ejecución: 0.8204185962677002


,idaviso,idpostulante,sepostulo
0,1111556097,EDp099,0.0
1,1111556097,NzewELB,0.0
2,1111556097,mzGKvNG,0.0
3,1111556097,5mlx0mN,0.0
4,1111556097,6J5YkM,0.0


In [69]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12837957 entries, 0 to 12837956
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
sepostulo       float64
dtypes: float64(1), int64(1), object(1)
memory usage: 391.8+ MB


Como podemos ver, obtuvimos una cantidad muy pareja de postulaciones y no postulaciones

In [70]:
modelo['sepostulo'].value_counts()

1.0    6603752
0.0    6234205
Name: sepostulo, dtype: int64

In [71]:
modelo['idaviso'] = modelo['idaviso'].astype('object')
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12837957 entries, 0 to 12837956
Data columns (total 3 columns):
idaviso         object
idpostulante    object
sepostulo       float64
dtypes: float64(1), object(2)
memory usage: 391.8+ MB


### Merge del modelo final con los detalles de los avisos

In [73]:
modelo = pd.merge(modelo, avisos_detalles, on=['idaviso'], how='inner')
modelo.head(4)

,idaviso,idpostulante,sepostulo,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,1111556097,EDp099,0.0,Responsable de Marketing-Producto Div. Constru...,3,10,3,Desarrollo de Negocios
1,1111556097,NzewELB,0.0,Responsable de Marketing-Producto Div. Constru...,3,10,3,Desarrollo de Negocios
2,1111556097,mzGKvNG,0.0,Responsable de Marketing-Producto Div. Constru...,3,10,3,Desarrollo de Negocios
3,1111556097,5mlx0mN,0.0,Responsable de Marketing-Producto Div. Constru...,3,10,3,Desarrollo de Negocios


In [74]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12274370 entries, 0 to 12274369
Data columns (total 8 columns):
idaviso            object
idpostulante       object
sepostulo          float64
titulo             object
nombre_zona        int64
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
dtypes: float64(1), int64(3), object(4)
memory usage: 842.8+ MB


Verificamos que el merge fue satisfactorio y no cuenta con valores nulos

In [75]:
(modelo['tipo_de_trabajo'].isnull().sum() == 0 , modelo['nivel_laboral'].isnull().sum() == 0)

(True, True)

### Merge del modelo final con la información de los postulantes

In [76]:
modelo = pd.merge(modelo, postulantes, on='idpostulante', how='inner')
modelo.head(4)

,idaviso,idpostulante,sepostulo,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,estudios,esta_estudiando,sexo,rango_edad
0,1111556097,EDp099,0.0,Responsable de Marketing-Producto Div. Constru...,3,10,3,Desarrollo de Negocios,6.0,0.0,2,4.0
1,1112425288,EDp099,0.0,EJECUTIVO DE VENTAS - CANAL FARMACEUTICO - GBA...,3,10,3,Comercial,6.0,0.0,2,4.0
2,1111901100,EDp099,0.0,Analista Contable Financiero Sr (Trilingüe),3,10,3,Finanzas Internacionales,6.0,0.0,2,4.0
3,1112231269,EDp099,0.0,Ejecutivo de ventas,3,10,3,Comercial,6.0,0.0,2,4.0


In [77]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12026515 entries, 0 to 12026514
Data columns (total 12 columns):
idaviso            object
idpostulante       object
sepostulo          float64
titulo             object
nombre_zona        int64
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
estudios           float64
esta_estudiando    float64
sexo               int64
rango_edad         float64
dtypes: float64(4), int64(4), object(4)
memory usage: 1.2+ GB


Verificamos que el merge fue satisfactorio y no cuenta con valores nulos

In [78]:
(modelo['estudios'].isnull().sum() == 0 , modelo['rango_edad'].isnull().sum() == 0)

(True, True)

Liberamos memoria para mejorar la performance

In [79]:
df_avisos_postulantes_all = 0
postulantes = 0
postulaciones = 0
avisos_detalles = 0

Analizamos la información del modelo final que será utilizado por los algoritmos de Machine Learning.

In [81]:
modelo.rename(columns={'estudios':'nivel_estudios'},inplace=True)

modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12026515 entries, 0 to 12026514
Data columns (total 12 columns):
idaviso            object
idpostulante       object
sepostulo          float64
titulo             object
nombre_zona        int64
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
nivel_estudios     float64
esta_estudiando    float64
sexo               int64
rango_edad         float64
dtypes: float64(4), int64(4), object(4)
memory usage: 1.2+ GB


Analizamos la cantidad de postulaciones y no postulaciones con las que cuenta el modelo

In [83]:
modelo['sepostulo'].value_counts()

0.0    6234205
1.0    5792310
Name: sepostulo, dtype: int64

Considerando la gran cantidad de registros que tiene el dataframe, decidimos tomar una porción aleatoria y equitativa de postulaciones y no postulaciones

In [84]:
no = modelo['sepostulo'] == 0
si = modelo['sepostulo'] == 1

modelo_no = modelo[no]
modelo_si = modelo[si]

In [85]:
n_samples = 500000

modelo_no = modelo_no.sample(n_samples)
modelo_si = modelo_si.sample(n_samples)

modelo_final = pd.concat([modelo_no, modelo_si])

modelo_final.sample(5)

,idaviso,idpostulante,sepostulo,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nivel_estudios,esta_estudiando,sexo,rango_edad
4169503,1112385104,xkaq1kb,0.0,"TO, Psicólogo, Psicopedagogo, Trabajador Socia...",3,10,3,Salud,4.0,0.0,2,2.0
4908575,1112377905,96zDb21,1.0,Vendedor/a para empresa financiera,3,10,2,Ventas,4.0,1.0,1,1.0
5331583,1112266297,JBXmxWr,1.0,Telemarketer - Call center en San Isidro - Bs....,2,9,1,Telemarketing,3.0,1.0,1,1.0
5360332,1112367910,owE8Vk4,1.0,Operario de depósito,3,10,1,Almacén / Depósito / Expedición,4.0,1.0,2,1.0
5365711,1112409567,zvPK1DD,0.0,Jefe de mantenimiento edilicio- con personal a...,3,10,3,Mantenimiento y Limpieza,3.0,0.0,1,3.0


In [86]:
modelo_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 2848002 to 7287200
Data columns (total 12 columns):
idaviso            1000000 non-null object
idpostulante       1000000 non-null object
sepostulo          1000000 non-null float64
titulo             1000000 non-null object
nombre_zona        1000000 non-null int64
tipo_de_trabajo    1000000 non-null int64
nivel_laboral      1000000 non-null int64
nombre_area        1000000 non-null object
nivel_estudios     1000000 non-null float64
esta_estudiando    1000000 non-null float64
sexo               1000000 non-null int64
rango_edad         1000000 non-null float64
dtypes: float64(4), int64(4), object(4)
memory usage: 99.2+ MB


Liberamos memoria para mejorar la performance

In [87]:
modelo_no = 0
modelo_si = 0

Reordenamos las columnas para facilitar el manejo del dataframe cuando apliquemos los algoritmos de ML

In [88]:
modelo_final = modelo_final[['idaviso','idpostulante','rango_edad','sexo',\
                             'nivel_estudios','esta_estudiando','titulo','tipo_de_trabajo',\
                             'nivel_laboral','nombre_zona','nombre_area','sepostulo']]

modelo_final.sample(5)

,idaviso,idpostulante,rango_edad,sexo,nivel_estudios,esta_estudiando,titulo,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,sepostulo
3967475,1112322158,13A4oN,2.0,1,4.0,1.0,Asistente legales,10,1,3,Legal,1.0
11539690,1112375329,1QPXxGN,4.0,2,6.0,1.0,PL Subgerente de Ventas Hipermercado Zona Pilar,10,4,3,Desarrollo de Negocios,0.0
5765693,1002498567,bOzrvZB,1.0,1,2.0,0.0,Vendedor para inmobiliaria,10,3,3,Ventas,0.0
3793333,1112284262,kPNd8G1,4.0,2,2.0,0.0,Operario de Producción - Plásticos,10,3,3,Producción,1.0
10070591,1112365828,EDM5Q4,3.0,1,6.0,1.0,Técnico Electromecanico para el área de Servic...,10,3,3,Programación de producción,0.0


Transformamos las variables categóricas del tipo entero, objeto o flotante a categóricas propiamente dichas. Esto mejora la performance notablemente.

In [89]:
modelo_final['rango_edad'] = modelo_final['rango_edad'].astype('category')
modelo_final['sexo'] = modelo_final['sexo'].astype('category') 
modelo_final['nivel_estudios'] = modelo_final['nivel_estudios'].astype('category')
modelo_final['esta_estudiando'] = modelo_final['esta_estudiando'].astype('category')
modelo_final['tipo_de_trabajo'] = modelo_final['tipo_de_trabajo'].astype('category')
modelo_final['nivel_laboral'] = modelo_final['nivel_laboral'].astype('category')
modelo_final['nombre_zona'] = modelo_final['nombre_zona'].astype('category')
modelo_final['sepostulo'] = modelo_final['sepostulo'].astype('category')

In [90]:
modelo_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 2848002 to 7287200
Data columns (total 12 columns):
idaviso            1000000 non-null object
idpostulante       1000000 non-null object
rango_edad         1000000 non-null category
sexo               1000000 non-null category
nivel_estudios     1000000 non-null category
esta_estudiando    1000000 non-null category
titulo             1000000 non-null object
tipo_de_trabajo    1000000 non-null category
nivel_laboral      1000000 non-null category
nombre_zona        1000000 non-null category
nombre_area        1000000 non-null object
sepostulo          1000000 non-null category
dtypes: category(8), object(4)
memory usage: 45.8+ MB


Exportamos el modelo final a un CSV file, para desentendernos de todas las operaciones costosas de dataframes a partir de este punto, y poder tener el set de entrenamiento ya listo para los algoritmos de Machine Learning.

In [91]:
modelo_final.to_csv("modelo_final.csv", encoding = "utf-8", index = False)